In [ ]:
%load_ext autoreload
%autoreload 2

import sys 
sys.path.insert(0, '..')
import jefpy as jp
import numpy as np
import matplotlib.pyplot as plt

%matplotlib qt

## Wires
`help(jp.Wire)`
Note the class methods for common wire configurations. 

### Circular wire

In [ ]:
osc = jp.HarmonicOscillator(phase=2.0, freq=300e6)
wire = jp.Wire.circle(I=osc.f, dI_dt=osc.df_dt)
jp.inspect_segments(wire.get_segmentation())
print(wire.E([0, 1, 2], 0), wire.B([0, 1, 2], 0))

In [ ]:
xrange, zrange = [-4.0, 4.0], [-4.0, 4.0]
surface_settings = {
    'x': np.linspace(*xrange, 80),
    'y': 0.5,
    'z': np.linspace(*zrange, 80)}
surface = jp.Surface.cartesian(**surface_settings)

observer = jp.Observer(surface.XYZ, wire, observable='norm')

visual = jp.MovieMap(observer.B)
visual.ax.set_xlabel('x (m)')
visual.ax.set_ylabel('z (m)')
visual.range = xrange + zrange
visual.live(slow_motion=1e-9, run_time=10)

## Coil

In [ ]:
num_windings = 20
width = 4

def coil(s, t):
    x = np.cos(2 * np. pi * num_windings * s)
    y = np.sin(2 * np. pi * num_windings * s)
    z = (s - 0.5) * width
    return np.array((x, y, z))

osc = jp.HarmonicOscillator(phase=2.0)
wire = jp.Wire.curve(coil, I=1.0, num_segments=200)
jp.inspect_segments(wire.get_segmentation())

In [ ]:
xrange, zrange = [-4.0, 4.0], [-4.0, 4.0]
surface_settings = {
    'x': np.linspace(*xrange, 80),
    'y': 0.5,
    'z': np.linspace(*zrange, 80)}
surface = jp.Surface.cartesian(**surface_settings)


B = wire.B(surface.XYZ)

plt.figure()
plt.streamplot(surface.u, surface.v, B[..., 0].T, B[..., 2].T)

### Flying Dipoles

In [ ]:
def location_1(t):
    return 3 * np.array([np.sin(80e6 * t), 0.0, np.cos(80e6 * t)])

dipole_1 = jp.MagneticDipole.oscillator(location=location_1, 
                                        m=(0, 0, 2), freq=200e6)

def location_2(t):
    return 3 * np.array([np.cos(40e6 * t), 0.0, np.sin(80e6 * t)])

dipole_2 = jp.MagneticDipole.oscillator(location=location_2, 
                                        m=(0, 0, 2), freq=200e6)

system = jp.SourceCollection((dipole_1, dipole_2))
                                   

In [ ]:
observer = jp.Observer(surface.XYZ, system, observable='norm')

visual = jp.MovieMap(observer.B)
visual.ax.set_xlabel('x (m)')
visual.ax.set_ylabel('z (m)')
visual.range = xrange + zrange
visual.live(slow_motion=5e-9, run_time=30)

## Curvelinear coordinates
- We consider sampling on curved surfaces
- Transforming the field in a different coordinate system
Take a look at: `help(jp.sampling)` and 

In [ ]:
osc = jp.HarmonicOscillator(freq=1e6, phase=0.5*np.pi)
poly = ([0, 0.0, -10], [0, 0.0, 0], [0, 0.0, 10])
wire = jp.Wire.polygon(poly, num_segments=100, I=osc.f, dI_dt=osc.df_dt)
jp.inspect_segments(wire.get_segmentation())

In [ ]:
phi = [0, 2* np.pi]
z = [-30, 30]
setup = {
    'phi': np.linspace(*phi, 20),
    'z': np.linspace(*z, 20),
    'radius': 1.0,
    'axis': 'z'}
surf = jp.Surface.cylinder(**setup)

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(*surf.XYZ.T)
plt.show()

In [ ]:
observer = jp.Observer(surf.XYZ, wire, observable=0)
visual = jp.MovieMap(observer.B)
visual.settings['aspect'] = 'auto'
visual.range = phi + z
visual.ax.set_xlabel("phi (rad)")
visual.ax.set_ylabel("z (m)")

visual.live(slow_motion=3E-7)

We are now looking at the x-coordinate on a cylinderical plane. This could be usefull in some cases, but typically one would like to transform the field as well to match the measurement plane.

In [ ]:
observer = jp.Observer(surf.XYZ, wire, transform=jp.to_cylindrical, observable=1)
visual = jp.MovieMap(observer.B)
visual.settings['aspect'] = 'auto'
visual.range = phi + z
visual.ax.set_xlabel("phi (rad)")
visual.ax.set_ylabel("z (m)")

visual.live(slow_motion=3E-7)

It is also possible of course to transform the field and plot it at one probe location.

In [ ]:
obs = jp.Observer([1, 10, 20], wire, transform=jp.to_spherical)
obs.B(np.linspace(0, 1e-6, 20))
time_series = jp.TimeSeries(obs.B)
time_series.ax[0].set_ylabel('B_r')
time_series.ax[1].set_ylabel('B_theta')
time_series.ax[2].set_ylabel('B_phi')
time_series(np.linspace(0, 1e-6, 20))

## Custom sources
Note that broadcast_spacetime is optional. It handles multi-dimensional calls. 

In [ ]:
class Custom(jp.Source):

    @jp.broadcast_spacetime
    def E(self, r, t):
        return [1, 0, 1.0] * np.cos(10 * t)
    
    @jp.broadcast_spacetime
    def B(self, r, t):
        return np.zeros(3) 

In [ ]:
source = Custom()
source.B([0, 1, 3], 5)

Of course such a source can be combined with probes and other sources in a Observation object. 